In [12]:
from tensorflow.examples.tutorials.mnist import input_data
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [10]:
mnist = input_data.read_data_sets("../data/", one_hot=True)

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [11]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f6227066908>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f62270665c0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f6227066860>)

In [13]:
# Hyperparameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [14]:
# TF Graph Input
x = tf.placeholder('float', [None, 784]) # Input size 28 x 28 images
y = tf.placeholder('float', [None, 10]) # -> Class Labels

In [25]:
# Creating Model

# Model Weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

with tf.name_scope('Wx_b') as scope:
    # Linear Model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
    
# Add summary ops to collect data - just for visualization purpose
w_h = tf.summary.histogram('weights', W)
b_h = tf.summary.histogram('biases', b)


with tf.name_scope('cost_function') as scope:
    cost_function = -tf.reduce_sum(y * tf.log(model))
    tf.summary.scalar('cost_function', cost_function)
    
with tf.name_scope('train') as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
    
    
# Initializing the variables
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter('../data/', graph_def=sess.graph_def)
    
    # Training Cycle
    for iteration in range(training_iteration):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            #Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys})
            
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y:batch_ys})/total_batch
            
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
            
        # Display logs per iteration step
        if iteration % display_step == 0:
            print('Iteration: ', str(iteration+1), 'cost= ', str(avg_cost))
            
    print('Training completed')
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y,1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, 'float'))
    print('Accuracy= ', accuracy.eval({x:mnist.test.images, y: mnist.test.labels}))

Iteration:  1 cost=  30.24644801746716
Iteration:  3 cost=  20.98726728179238
Iteration:  5 cost=  20.14931133270263
Iteration:  7 cost=  19.69932788068598
Iteration:  9 cost=  19.286752523942432
Iteration:  11 cost=  19.115124676444317
Iteration:  13 cost=  18.908807949586357
Iteration:  15 cost=  18.771979695233423
Iteration:  17 cost=  18.58577195601029
Iteration:  19 cost=  18.56238771872086
Iteration:  21 cost=  18.48087764219804
Iteration:  23 cost=  18.350156127756296
